In [ ]:
# !pip install transformers datasets jiwer PrettyTable openai-whisper ffmpeg pydub

# Check CUDA

In [1]:
import torch
import warnings

print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device: {cuda_id}")

print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

warnings.filterwarnings("ignore")

Is CUDA supported by this system? True
CUDA version: 11.8
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce RTX 3050 Laptop GPU


# Load model

In [33]:
from transformers import pipeline

#### Whisper model

In [34]:
whisper = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3", device='cuda')

#### PhoWhisper model

In [35]:
pho_whisper = pipeline("automatic-speech-recognition", model="vinai/PhoWhisper-large", device='cuda')

#### Wav2Vec model

In [36]:
wav2vec = pipeline("automatic-speech-recognition", model="nguyenvulebinh/wav2vec2-base-vietnamese-250h", device='cuda')

c:\ProgramData\miniforge3\envs\speech2text\lib\site-packages\transformers\configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at nguyenvulebinh/wav2vec2-base-vietnamese-250h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identica

# Load Data

In [6]:
import os
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('valTest.csv')
valTest = []
for index, row in df.iterrows():
  obj = dict()
  obj['text'] = row['text']
  obj['file'] = row['file']
  obj['path'] = row['path']
  valTest.append(obj)

In [8]:
len(valTest)

6210

# Evaluate

In [9]:
from prettytable import PrettyTable
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [10]:
result = []
for test in tqdm(valTest):
  row = []
  row.append(test['text'])
  row.append(whisper(test['path'])['text'])
  row.append(pho_whisper(test['path'])['text'])
  row.append(wav2vec(test['path'])['text'])
  result.append(row)

  0%|          | 0/6210 [00:00<?, ?it/s]Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
  0%|          | 25/6210 [1:52:28<463:47:01, 269.95s/it]


In [ ]:
df = pd.DataFrame(result, columns=['Actual', "Whisper", "PhoWhisper", "Wav2Vec"])
df.to_csv('results.csv', index=False)

## Create metric

In [16]:
import jiwer
import pandas as pd
from prettytable import PrettyTable

In [6]:
df = pd.read_csv('results.csv')

In [23]:
len(df)

6210

In [10]:
whisper_wer = 0
whisper_cer = 0
for ref, hyp in zip(df['Actual'], df['Whisper']):
  whisper_wer += jiwer.wer(ref, hyp)
  whisper_cer += jiwer.cer(ref, hyp)

In [11]:
pho_whisper_wer = 0
pho_whisper_cer = 0
for ref, hyp in zip(df['Actual'], df['PhoWhisper']):
  pho_whisper_wer += jiwer.wer(ref, hyp)
  pho_whisper_cer += jiwer.cer(ref, hyp)

In [12]:
wav2vec_wer = 0
wav2vec_cer = 0
for ref, hyp in zip(df['Actual'], df['Wav2Vec']):
  wav2vec_wer += jiwer.wer(ref, hyp)
  wav2vec_cer += jiwer.cer(ref, hyp)

In [21]:
# Create tabel for WER and CER
table = PrettyTable()
table.field_names = ["Model", "WER", "CER"]
table.add_row(["Whisper", round(whisper_wer/len(df), 3), round(whisper_cer/len(df), 3)])
table.add_row(["PhoWhisper", round(pho_whisper_wer/len(df),3), round(pho_whisper_cer/len(df),3)])
table.add_row(["Wav2Vec", round(wav2vec_wer/len(df),3), round(wav2vec_cer/len(df),3)])

In [22]:
table

Model,WER,CER
Whisper,0.776,0.606
PhoWhisper,0.26,0.203
Wav2Vec,0.272,0.199


In [28]:
tab = pd.DataFrame([["Whisper", round(whisper_wer/len(df), 3), round(whisper_cer/len(df), 3)],
                    ["PhoWhisper", round(pho_whisper_wer/len(df),3), round(pho_whisper_cer/len(df),3)],
                    ["Wav2Vec", round(wav2vec_wer/len(df),3), round(wav2vec_cer/len(df),3)]], columns=['Model', 'WER', 'CER'])

In [30]:
tab.to_csv('wer_cer_metric.csv', index=False)

## Random sample

In [31]:
import random

In [38]:
for _ in range(5):
  num = random.randint(0, len(df))
  print(f"Sample: {_+1}")
  print(f"Actual: {df['Actual'][num]}")
  print(f"Whisper: {df['Whisper'][num]}")
  print(f"PhoWhisper: {df['PhoWhisper'][num]}")
  print(f"Wav2Vec: {df['Wav2Vec'][num]}")
  print()

Sample: 1
Actual: không che mặt và tay chân của trẻ tùy thời tiết cho phép mẹ trong thai kỳ nên
Whisper:  Hãy subscribe cho kênh Ghiền Mì Gõ Để không bỏ lỡ những video hấp dẫn
PhoWhisper: không che mặt và tay chân của trẻ tuỳ thời tiết cho phép mẹ trong thai kỳ.
Wav2Vec: không che mặt và tay chân của trẻ tùy thời tiết cho phép mẹ trong thai kỳ

Sample: 2
Actual: rồi thứ hai lên đừng có kỳ thị tôi quá hãy coi tôi cũng giống như những người khác thôi chứ đừng có cho là tôi là mập
Whisper:  Rồi thứ hai là đừng có kỳ thị tôi quá Hãy coi tôi cũng giống như những người khác thôi Chứ đừng có cho tôi là mập địch ú
PhoWhisper: rồi thứ hai là đừng có kỳ thị tôi quá hãy coi tôi cũng giống như những người khác thôi chớ đừng có cho là tôi là mập địch uống.
Wav2Vec: rồi thứ hai là đừng có kỳ thị tôi quá hay coi tôi là cũng giống như những người khác thôi chứ đừng có cho là tôi là mập địch uống

Sample: 3
Actual: người bạn đồng hành đáng tin cậy mà lại rất rẻ tiền rất là hợp lý đúng không ạ cái thứ h